<a href="https://colab.research.google.com/github/crashidian/RandomScriptsForCS/blob/master/SchoolFacilitiesOptionsDevelopment_D2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import numpy as np
import pandas as pd
import torch
import torch_geometric
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, Batch
from sklearn.preprocessing import StandardScaler

In [67]:
class SchoolFacility:
    def __init__(self, name, condition, capacity, current_enrollment, age, annual_maintenance_cost, location):
        self.name = name
        self.condition = condition  # 0-100 scale
        self.capacity = capacity
        self.current_enrollment = current_enrollment
        self.age = age
        self.annual_maintenance_cost = annual_maintenance_cost
        self.location = location  # (lat, lon)

In [68]:
class SchoolDistrict:
    def __init__(self, name):
        self.name = name
        self.schools = []
        self.population_growth_rate = 0.01  # 1% annual growth
        self.bond_amount = 0

In [69]:
class Project:
    def __init__(self, name, school, capital_cost, annual_maintenance_impact, capacity_change, condition_improvement):
        self.name = name
        self.school = school
        self.capital_cost = capital_cost
        self.annual_maintenance_impact = annual_maintenance_impact
        self.capacity_change = capacity_change
        self.condition_improvement = condition_improvement

In [70]:
class Option:
    def __init__(self, name, projects):
        self.name = name
        self.projects = projects
        self.total_capital_cost = sum(project.capital_cost for project in projects)
        self.total_maintenance_impact = sum(project.annual_maintenance_impact for project in projects)
        self.total_capacity_change = sum(project.capacity_change for project in projects)
        self.average_condition_improvement = np.mean([project.condition_improvement for project in projects])

In [71]:
class GNNModel(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = torch.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

In [72]:
class GNNEnhancedPlanningSystem:
    def __init__(self):
        self.district = None
        self.gnn_model = None

    def load_district_data(self, district):
        self.district = district

    def create_graph(self):
        node_features = []
        for school in self.district.schools:
            node_features.append([
                school.condition / 100,  # Normalize to 0-1 range
                school.capacity / 2000,  # Assuming max capacity is 2000
                school.current_enrollment / 2000,
                school.age / 100,  # Assuming max age is 100 years
                school.annual_maintenance_cost / 1000000,  # Normalize to millions
                (school.location[0] + 90) / 180,  # Normalize latitude
                (school.location[1] + 180) / 360  # Normalize longitude
            ])

        edges = []
        for i in range(len(self.district.schools)):
            for j in range(i+1, len(self.district.schools)):
                dist = self.calculate_distance(self.district.schools[i].location, self.district.schools[j].location)
                if dist < 10:  # Assuming schools within 10 km are connected
                    edges.append([i, j])
                    edges.append([j, i])

        x = torch.tensor(node_features, dtype=torch.float)
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

        return Data(x=x, edge_index=edge_index)

    def calculate_distance(self, loc1, loc2):
        return ((loc1[0] - loc2[0])**2 + (loc1[1] - loc2[1])**2)**0.5

    def train_gnn(self, graph):
        self.gnn_model = GNNModel(num_node_features=7, num_classes=4)
        optimizer = torch.optim.Adam(self.gnn_model.parameters(), lr=0.01)

        self.gnn_model.train()
        for epoch in range(200):
            optimizer.zero_grad()
            out = self.gnn_model(graph)
            # Create target based on current conditions (just for demonstration)
            target = torch.tensor([0 if school.condition < 50 else
                                   1 if school.condition < 70 else
                                   2 if school.condition < 90 else 3
                                   for school in self.district.schools], dtype=torch.long)
            loss = torch.nn.functional.cross_entropy(out, target)
            loss.backward()
            optimizer.step()

    def generate_project_options(self):
        graph = self.create_graph()
        self.train_gnn(graph)

        with torch.no_grad():
            node_embeddings = self.gnn_model(graph)

        options = []

        # Generate options based on GNN output
        for i in range(4):  # Generate 4 distinct options
            option_projects = []
            for j, school in enumerate(self.district.schools):
                project_type = torch.argmax(node_embeddings[j]).item()

                if project_type == 0:  # Minimal intervention
                    option_projects.append(Project(f"Essential repairs for {school.name}", school,
                                                   capital_cost=2000000,
                                                   annual_maintenance_impact=-50000,
                                                   capacity_change=0,
                                                   condition_improvement=20))
                elif project_type == 1:  # Moderate renovation
                    option_projects.append(Project(f"Moderate renovation of {school.name}", school,
                                                   capital_cost=10000000,
                                                   annual_maintenance_impact=-100000,
                                                   capacity_change=50,
                                                   condition_improvement=40))
                elif project_type == 2:  # Comprehensive modernization
                    option_projects.append(Project(f"Comprehensive modernization of {school.name}", school,
                                                   capital_cost=25000000,
                                                   annual_maintenance_impact=-200000,
                                                   capacity_change=200,
                                                   condition_improvement=80))
                else:  # New construction
                    option_projects.append(Project(f"Replace {school.name} with new building", school,
                                                   capital_cost=40000000,
                                                   annual_maintenance_impact=-300000,
                                                   capacity_change=500,
                                                   condition_improvement=100))

            options.append(Option(f"GNN-Optimized Option {i+1}", option_projects))

        return options

    def rank_options(self, options):
        def calculate_score(option):
            capital_cost_score = 1 / (option.total_capital_cost / 1e6)
            maintenance_score = -option.total_maintenance_impact / 1e6
            capacity_score = option.total_capacity_change / 100
            condition_score = option.average_condition_improvement / 10

            return (capital_cost_score * 0.3 +
                    maintenance_score * 0.2 +
                    capacity_score * 0.3 +
                    condition_score * 0.2)

        ranked_options = sorted(options, key=calculate_score, reverse=True)
        return ranked_options

In [73]:
def main():
    # Create a sample school district
    district = SchoolDistrict("Sample Unified School District")
    district.schools = [
        SchoolFacility("High School A", 65, 1500, 1400, 30, 1000000, (34.0522, -118.2437)),
        SchoolFacility("Middle School B", 55, 1000, 950, 40, 800000, (34.0689, -118.4452)),
        SchoolFacility("Elementary School C", 80, 600, 550, 15, 500000, (34.1478, -118.1445)),
        SchoolFacility("Elementary School D", 45, 500, 480, 55, 600000, (33.9416, -118.4085)),
    ]
    district.bond_amount = 100000000  # $100 million bond

    # Initialize and run the GNN-enhanced planning system
    gnn_system = GNNEnhancedPlanningSystem()
    gnn_system.load_district_data(district)

    options = gnn_system.generate_project_options()
    ranked_options = gnn_system.rank_options(options)

    # Print results
    print(f"GNN-Enhanced Multi-Option Facility Planning for {district.name}")
    print(f"Total Bond Amount Available: ${district.bond_amount:,}")
    print("\nRanked Options (from most viable to least viable):")

    for i, option in enumerate(ranked_options, 1):
        print(f"\n{i}. {option.name}")
        print(f"   Total Capital Cost: ${option.total_capital_cost:,}")
        print(f"   Annual Maintenance Impact: ${option.total_maintenance_impact:,}")
        print(f"   Total Capacity Change: {option.total_capacity_change}")
        print(f"   Average Condition Improvement: {option.average_condition_improvement:.2f}")
        print("   Projects:")
        for project in option.projects:
            print(f"   - {project.name}")
            print(f"     Capital Cost: ${project.capital_cost:,}")
            print(f"     Annual Maintenance Impact: ${project.annual_maintenance_impact:,}")
            print(f"     Capacity Change: {project.capacity_change}")
            print(f"     Condition Improvement: {project.condition_improvement}")

    # Provide a recommendation
    best_option = ranked_options[0]
    print(f"\nRecommendation:")
    print(f"Based on the GNN analysis, the most viable option is: {best_option.name}")
    print(f"This option provides the best balance of capital costs, maintenance impact, capacity improvement, and overall condition enhancement.")
    if best_option.total_capital_cost > district.bond_amount:
        print(f"Note: This option exceeds the available bond amount by ${best_option.total_capital_cost - district.bond_amount:,}.")
        print("Consider phasing the projects or seeking additional funding sources.")
    else:
        print(f"This option is within the available bond amount, with ${district.bond_amount - best_option.total_capital_cost:,} remaining for contingencies or additional improvements.")

In [74]:
if __name__ == "__main__":
    main()

GNN-Enhanced Multi-Option Facility Planning for Sample Unified School District
Total Bond Amount Available: $100,000,000

Ranked Options (from most viable to least viable):

1. GNN-Optimized Option 1
   Total Capital Cost: $40,000,000
   Annual Maintenance Impact: $-400,000
   Total Capacity Change: 200
   Average Condition Improvement: 40.00
   Projects:
   - Moderate renovation of High School A
     Capital Cost: $10,000,000
     Annual Maintenance Impact: $-100,000
     Capacity Change: 50
     Condition Improvement: 40
   - Moderate renovation of Middle School B
     Capital Cost: $10,000,000
     Annual Maintenance Impact: $-100,000
     Capacity Change: 50
     Condition Improvement: 40
   - Moderate renovation of Elementary School C
     Capital Cost: $10,000,000
     Annual Maintenance Impact: $-100,000
     Capacity Change: 50
     Condition Improvement: 40
   - Moderate renovation of Elementary School D
     Capital Cost: $10,000,000
     Annual Maintenance Impact: $-100,000
